In [3]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
import os
path=os.getcwd()
print(path)

/content


In [4]:
data = pd.read_csv('/spam.csv',encoding='latin-1') 
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":"label", "v2":"text"})
print(data.describe())
data.groupby("label").describe()

       label                    text
count   5572                    5572
unique     2                    5169
top      ham  Sorry, I'll call later
freq    4825                      30


text                                                               
      count unique                                                top freq
label                                                                     
ham    4825   4516                             Sorry, I'll call later   30
spam    747    653  Please call our customer service representativ...    4

In [5]:
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
import os
path=os.getcwd()
print(path)

/content


In [7]:
#The dataset contains 4825 ham and 747 spam messages.
#For both classes, some messages appear more than once (common phrases, etc.).

data.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [8]:
#Add numerical label for spam
#Target must be numerical for ML classification models
data['spam'] = data['label'].map( {'spam': 1, 'ham': 0} ).astype(int)
data.head(15)

,label,text,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
6,ham,Even my brother is not like to speak with me. ...,0
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1


In [10]:
#Add feature: length of message
data['length'] = data['text'].apply(len)
data.head(10)

,label,text,spam,length
0,ham,"Go until jurong point, crazy.. Available only ...",0,111
1,ham,Ok lar... Joking wif u oni...,0,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155
3,ham,U dun say so early hor... U c already then say...,0,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1,148
6,ham,Even my brother is not like to speak with me. ...,0,77
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0,160
8,spam,WINNER!! As a valued network customer you have...,1,158
9,spam,Had your mobile 11 months or more? U R entitle...,1,154


In [9]:
data_ham  = data[data['spam'] == 0].copy()
data_spam = data[data['spam'] == 1].copy()

In [11]:
#Remove Punctuation and Stopwords
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
def remove_punctuation_and_stopwords(sms):
    
    sms_no_punctuation = [ch for ch in sms if ch not in string.punctuation]
    sms_no_punctuation = "".join(sms_no_punctuation).split()
    
    sms_no_punctuation_no_stopwords = \
        [word.lower() for word in sms_no_punctuation if word.lower() not in stopwords]
        
    return sms_no_punctuation_no_stopwords

In [13]:
data['text'].apply(remove_punctuation_and_stopwords).head()

0    [go, jurong, point, crazy, available, bugis, n...
1                       [ok, lar, joking, wif, u, oni]
2    [free, entry, 2, wkly, comp, win, fa, cup, fin...
3        [u, dun, say, early, hor, u, c, already, say]
4    [nah, dont, think, goes, usf, lives, around, t...
Name: text, dtype: object

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer = remove_punctuation_and_stopwords).fit(data['text'])
print(len(bow_transformer.vocabulary_))

9449


In [15]:
bow_data = bow_transformer.transform(data['text'])
bow_data.shape

(5572, 9449)

##Analysis

To analyze the text data, we have to turn the words into numerical numbers. 
We have multiple choices to accomplish this step: 

1) Binary Term Frequency :  count presence(1) or absence(0) for term in document

2) Bag of Words Frequency:  captures the frequency of term in document

3) Term Frequency: 

4) TFIDF :

in this way, if a term appears frequently in a document, it’s important; if a term appears in many documents, it’s not a unique identifier.

Word2Vec.

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(bow_data)

sample_ham = data['text'][4]
bow_sample_ham = bow_transformer.transform([sample_ham])
tfidf_sample_ham = tfidf_transformer.transform(bow_sample_ham)
print(tfidf_sample_ham)
print('---------------------------------------')
sample_spam = data['text'][8]
bow_sample_spam = bow_transformer.transform([sample_spam])
tfidf_sample_spam = tfidf_transformer.transform(bow_sample_spam)
print(tfidf_sample_spam)

  (0, 8679)	0.4000611457714059
  (0, 8258)	0.35462891367407595
  (0, 8239)	0.2665748147939407
  (0, 5638)	0.40493594386802356
  (0, 5008)	0.44910913790937335
  (0, 3764)	0.35674330934716636
  (0, 2884)	0.23016324066220836
  (0, 1279)	0.31179198007951625
---------------------------------------
  (0, 9436)	0.2604683359422661
  (0, 9045)	0.23445094204464131
  (0, 8713)	0.24115248419490207
  (0, 8709)	0.22254833340975305
  (0, 7223)	0.2148462495005997
  (0, 6959)	0.2564029044370425
  (0, 6788)	0.2943229069695555
  (0, 6550)	0.178899142428953
  (0, 5714)	0.2148462495005997
  (0, 4763)	0.2943229069695555
  (0, 4197)	0.22555164191980467
  (0, 2531)	0.1915015107445745
  (0, 2246)	0.21371446699888028
  (0, 2185)	0.3406301232141347
  (0, 1906)	0.1153380143353214
  (0, 323)	0.2387711073538672
  (0, 218)	0.2943229069695555


#Next we use CountVectorizer:

More Details and example at:

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [ ]:
#MINOR CORRECTION IN LAB'S CODE:

#BY MISTAKE WE HAVE APPLIED "FIT_TRANSFORM" TO TEST DATA ALSO INSTEAD OF ONLY "TRANSFORM"
#AND WHENEVER WE HAVE CORRECTED IT, THE XTRAIN,XTEST WAS ALREADY MODIFIED
#SO, SIMPLE RERUN YOUR TRAIN_TEST_SPLIT CODE AND NEXT IF YOU TRY WITH THE GIVEN CODE, IT WILL WORK

#Hope You got the Point !!!!!!

In [17]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

**Naive Bayes**

In [18]:
#import the necessary module
from sklearn.model_selection import train_test_split

data_tfidf = tfidf_transformer.transform(bow_data)

#split data set into train and test sets
data_tfidf_train, data_tfidf_test, label_train, label_test = \
    train_test_split(data_tfidf, data["spam"], test_size=0.3, random_state=68)



In [19]:
data_tfidf_train = data_tfidf_train.A
data_tfidf_test = data_tfidf_test.A

from sklearn.naive_bayes import GaussianNB, MultinomialNB

#Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics  import accuracy_score

spam_detect_model = GaussianNB().fit(data_tfidf_train, label_train)
pred_test_MNB = spam_detect_model.predict(data_tfidf_test)
acc_MNB = accuracy_score(label_test, pred_test_MNB)
print(acc_MNB)

0.9037081339712919


**Decision Tree**

In [21]:
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
#Create a Decision Tree Classifier (using Entropy)
clf_Entropy=DecisionTreeClassifier(criterion="entropy")

# Train the model using the training sets
clf_Entropy.fit(data_tfidf_train, label_train)
pred_test_MNB = clf_entropy.predict(data_tfidf_test)
acc_MNB = accuracy_score(label_test, pred_test_MNB)
print(acc_MNB)

0.9599282296650717


**Optional Exercise:**
Try this on full spam.csv file and bigram matching instead of unigram matching 